# 🎨 ST-Visions Live Demo 2: Data Colorization

Visualizing spatial-temporal data becomes far more effective when color encodes meaning. In this demo, we demonstrate how to use ST_Visions' built-in colorization features to highlight patterns in a given geospatial dataset. The notebook is organized in Use Cases depending on what we are trying to or what kind of visualization problem we are trying to solve

The library has several prebeuilt palettes for both numerical and categorical colormaps, and supports custom palettes 

In [ ]:
import sys, os

# ===== DEV ENVIROMENT CONFIGURATION (TEMPORARY) =====

%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../..')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

# =====================================================

import sys, os
import pandas as pd
import bokeh.models as bokeh_models
import bokeh.palettes as bokeh_palettes

from st_visions.st_visualizer import st_visualizer
import st_visions.express as viz_express
import st_visions.geom_helper as viz_helper
import st_visions.providers as viz_providers

## Use-Case #1: Colormapping a **Single Feature**.

### Categorical feature Colormapping

**NOTE:** The Categorical Feature must be in String form. 

In this example, we showcase how to assign unique colors to categorical values using Bokeh's CategoricalColorMapper. We load a dummy dataset of global airports, geospatially processed with latitude and longitude coordinates. 

In [ ]:
world_airports = pd.read_csv(r'..\..\data\airports_dummy.csv')
world_airports = viz_helper.create_geometry(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs='epsg:4326')
world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)].copy()
world_airports = world_airports.loc[world_airports.Altitude >= 0].copy()

We categorize the airports by the City_Country field — each unique location is assigned a distinct color.

In [ ]:
st_viz = st_visualizer(limit=5000)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='fixed', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]

st_viz.add_categorical_colormap('Category20b', 'City_Country')
_ = st_viz.add_marker(marker='circle', fill_color=st_viz.cmap, line_color=st_viz.cmap, alpha=1, legend_group='City_Country')

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)

### Bonus: Handling a large amount of categorical values

Assuming a categorical colormap based on vessel_id feature, there is a considerable probability that the amount of unique vessel_id records in a specific time snapshot are more than the maximum size of a given categorical color palette. In that case the categorical colormap function cycles the available colors until all the unique categorical recrods are covered. Due to the nature of the data (vessel movement) the colorization of the vessel_Ids in this way does not obstruct much of the "story" of the figure 

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_categorical_colormap(palette='Category20b', categorical_name='vessel_id')
circ = st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')


### Numerical Feature Colormapping

In the following example, we demonstrate how to colorize point-based data using a numerical feature — specifically, vessel speed. To do so, we apply a LinearColorMapper with a built in numerical colormap function

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='fixed', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
_ = st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'Speed Heatmap')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)


## Use-Case #2 Colormap & Filter Interactions

In this use case, we demonstrate how colormaps can interact and mesh with other filters provided by the library to offer more robust insights 

### Combining a Numerical Colormap with a Temporal Filter

This example demonstrates how to map a numerical feature to color using Bokeh's LinearColorMapper, while simultaneously enabling temporal filtering to explore changes over time.

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'GPS Locations (Speed Heatmap)')
st_viz.add_temporal_filter(callback_policy='value', temporal_name='t', temporal_unit='ms')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)

### Combining a Categorical Colormap with a Temporal Filter

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_categorical_colormap(palette=bokeh_palettes.Category20c_20, categorical_name='vessel_id')
st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'Vessel Positions')
st_viz.add_temporal_filter(callback_policy='value', step_ms=1000, temporal_name='t', temporal_unit='ms')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')